<a href="https://colab.research.google.com/github/DrAlexSanz/NLP-SPEC-C4/blob/main/W1/Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from termcolor import colored
import random
import numpy as np
!pip install trax
import trax
from trax import layers as tl
from trax.fastmath import numpy as fastnp
from trax.supervised import training

!pip list | grep trax

     |████████████████████████████████| 471kB 12.9MB/s 
     |████████████████████████████████| 2.6MB 51.1MB/s 
     |████████████████████████████████| 174kB 57.5MB/s 
     |████████████████████████████████| 1.1MB 51.1MB/s 
     |████████████████████████████████| 348kB 49.5MB/s 
     |████████████████████████████████| 71kB 11.8MB/s 
     |████████████████████████████████| 1.4MB 55.5MB/s 
     |████████████████████████████████| 3.7MB 45.7MB/s 
     |████████████████████████████████| 890kB 49.5MB/s 
     |████████████████████████████████| 2.9MB 42.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=46730cd3f7bd43ef61076e887aa93d0a65e039a82de52e9a9ef114918f01c1a5
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses


trax                          1.3.6                


In [3]:
train_stream_fn = trax.data.TFDS('opus/medical',
                                 data_dir='./data/',
                                 keys=('en', 'de'),
                                 eval_holdout_size=0.01, # 1% for eval
                                 train=True)

# Get generator function for the eval set
eval_stream_fn = trax.data.TFDS('opus/medical',
                                data_dir='./data/',
                                keys=('en', 'de'),
                                eval_holdout_size=0.01, # 1% for eval
                                train=False)

Dataset opus downloaded and prepared to ./data/opus/medical/0.1.0. Subsequent calls will reuse this data.


In [ ]:
!git clone https://github.com/DrAlexSanz/NLP-SPEC-C4.git
!cp "NLP-SPEC-C4/W1/data/ende_32k.subword" "data"
!rm -rf "NLP-SPEC-C4"


In [ ]:
# global variables that state the filename and directory of the vocabulary file
VOCAB_FILE = 'ende_32k.subword'
VOCAB_DIR = 'data/'